# Black-Scholes Option Pricing Model

Black–Scholes–Merton model is a mathematical model for the dynamics of a financial market containing derivative investment instruments. From the partial differential equation in the model, known as the Black–Scholes equation, one can deduce the Black–Scholes formula, which gives a theoretical estimate of the price of European-style options and shows that the option has a unique price regardless of the risk of the security and its expected return (instead replacing the security's expected return with the risk-neutral rate).

The key idea behind the model is to hedge the option by buying and selling the underlying asset in just the right way and, as a consequence, to eliminate risk. This type of hedging is called "continuously revised delta hedging" and is the basis of more complicated hedging strategies such as those engaged in by investment banks and hedge funds.

The Black–Scholes model assumes that the market consists of at least one risky asset, usually called the stock, and one riskless asset, usually called the money market, cash, or bond.

Now we make assumptions on the assets (which explain their names):

- (riskless rate) The rate of return on the riskless asset is constant and thus called the risk-free interest rate.
- (random walk) The instantaneous log return of stock price is an infinitesimal random walk with drift; more precisely, it is a geometric Brownian motion, and we will assume its drift and volatility are constant (if they are time-varying, we can deduce a suitably modified Black–Scholes formula quite simply, as long as the volatility is not random).
- The stock does not pay a dividend.

Assumptions on the market:

- There is no arbitrage opportunity (i.e., there is no way to make a riskless profit).
- It is possible to borrow and lend any amount, even fractional, of cash at the riskless rate.
- It is possible to buy and sell any amount, even fractional, of the stock (this includes short selling).
- The above transactions do not incur any fees or costs (i.e., frictionless market).


The Black-Scholes formula:

\begin{equation}
\frac{\partial V}{ \partial t} + \frac{1}{2}\sigma^{2} S^{2} \frac{\partial^{2} V}{\partial S^2} + r S \frac{\partial V}{\partial S} - r V = 0
\end{equation}

"The Greeks" measure the sensitivity of the value of a derivative or a portfolio to changes in parameter value(s) while holding the other parameters fixed. They are partial derivatives of the price with respect to the parameter values. One Greek, "gamma" is a partial derivative of another Greek, "delta" in this case.

The Greeks are important not only in the mathematical theory of finance, but also for those actively trading. Financial institutions will typically set (risk) limit values for each of the Greeks that their traders must not exceed. Delta is the most important Greek since this usually confers the largest risk. Many traders will zero their delta at the end of the day if they are speculating and following a delta-neutral hedging approach as defined by Black–Scholes.
Typically the following Greeks are of interest: Delta, Vega and Gamma.

Black-Scholes Call and Put Option Price Formulas:

$C=S_{0}*N(d_{1})-Ke^{-rt}*N(d_{2})$

$P=Ke^{-rt}*N(-d_{2})-S_{0}*N(-d_{1})$

where $N(x)$ is the standard normal cumulative distribution function

$d_{1}=\frac{\ln{\frac{S_{o}}{K}}+t(r+\frac{\sigma^{2}}{2})}{\sigma\sqrt{t}}$

$d_{2}=d_{1}-\sigma\sqrt{t}$

In [5]:
import math
from scipy import stats

In [6]:
# Option parameters

K = 110 # strike price
r = .04 # interest rate
sigma = .3 # volatility - standard deviation of the stock
T = 1 # time maturity
S0 = 100 # starting price of the stock

In [7]:
def get_black_scholes_simulation_results(K, r, sigma, T, S0, is_call=True):
    """
    Calculates option price using Black-Scholes model, 
    returns the option value and the dict of calculated greeks in the form of a list
    """
    d1 = (math.log(S0 / K) + 
          (r + (0.5 * sigma**2)) * T) / (sigma * math.sqrt(T))
    d2 = d1 - (sigma * math.sqrt(T))

    if is_call:
        call_value = S0 * stats.norm.cdf(d1, 0, 1) - K * math.exp(
            -r * T) * stats.norm.cdf(d2, 0, 1)
        delta_call = stats.norm.cdf(d1, 0, 1)
        gamma_call = stats.norm.pdf(d1, 0, 1) / (S0 * sigma * math.sqrt(T))
        theta_call = -(r * K * math.exp(-r * T) * stats.norm.cdf(d2, 0, 1)) - (
            sigma * S0 * stats.norm.pdf(d1, 0, 1) / (2 * math.sqrt(T)))
        rho_call = T * K * math.exp(-r * T) * stats.norm.cdf(d2, 0, 1)
        vega_call = math.sqrt(T) * S0 * stats.norm.pdf(d1, 0, 1)

        return [
            call_value, {
                'delta': delta_call,
                'gamma': gamma_call,
                'theta': theta_call,
                'rho': rho_call,
                'vega': vega_call
            }
        ]

    else:
        put_value = K * math.exp(-r * T) * stats.norm.cdf(-d2, 0, 1) - (
            S0 * stats.norm.cdf(-d1, 0, 1))
        delta_put = -stats.norm.cdf(-d1, 0, 1)
        gamma_put = stats.norm.pdf(d1, 0, 1) / (S0 * sigma * math.sqrt(T))
        theta_put = (r * K * math.exp(-r * T) * stats.norm.cdf(-d2, 0, 1)) - (
            sigma * S0 * stats.norm.pdf(d1, 0, 1) / (2 * math.sqrt(T)))
        rho_put = -T * K * math.exp(-r * T) * stats.norm.cdf(-d2, 0, 1)
        vega_put = math.sqrt(T) * S0 * stats.norm.pdf(d1, 0, 1)

        return [
            put_value, {
                'delta': delta_put,
                'gamma': gamma_put,
                'theta': theta_put,
                'rho': rho_put,
                'vega': vega_put
            }
        ]

In [9]:
bs_call_results = get_black_scholes_simulation_results(K, r, sigma, T, S0)
bs_put_results = get_black_scholes_simulation_results(K, r, sigma, T, S0, is_call=False)

print('\nVanila European Call:\n' + '\nValue: ' + str(bs_call_results[0]) + '\nDelta: ' +
      str(bs_call_results[1]['delta']))
print('\nVanila European Put:\n' + '\nValue: ' + str(bs_put_results[0]) + '\nDelta: ' +
      str(bs_put_results[1]['delta']))


Vanila European Call:

Value: 9.625357828843697
Delta: 0.48629214299030143

Vanila European Put:

Value: 15.312196135599244
Delta: -0.5137078570096986
